In [ ]:
baseline3_split_dataset

In [ ]:
由于split的dataset可以固定了，因此就将几个split后的dataset保存起来
然后用baseline3执行就好了

In [ ]:
注：这里对B也要处理

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_x = pd.read_csv('dataset/train/ord_train_pre2.csv')
test_A = pd.read_csv('dataset/test/ord_testA_pre2.csv',encoding = 'GB2312')
test_B = pd.read_csv('dataset/test/ord_testB_pre2.csv',encoding = 'GB2312')

In [3]:
train_x.shape,test_A.shape,test_B.shape

((4023086, 22), (11035, 22), (11036, 22))

In [ ]:
date类型转换（没法保存，必须每次都做）

In [4]:
from dateutil.parser import parse
train_x['orderdate'] = train_x['orderdate'].apply(parse)
test_A['orderdate'] = test_A['orderdate'].apply(parse)
test_B['orderdate'] = test_B['orderdate'].apply(parse)

In [ ]:
出现次数少的是否要处理？
——处理，但是必须非常低的，比如1,2次，10次不行

In [ ]:
先处理下nan
——要处理的只有zone

In [ ]:
train_x.isnull().sum()

In [ ]:
-1,0,nan三者一个意思
——全用-1代替

In [5]:
train_x['zone'].replace(np.nan,-1,inplace = True)
train_x['zone'].replace(0,-1,inplace = True)

In [ ]:
需要考虑去除低频id的特征：
countryid 只有202个，暂时不去除；
city 12000+，去除出现次数小于3次的
supplierid，700个左右，不处理
zone 3000左右，不处理（=1的300个）

In [6]:
cityid = train_x['city'].value_counts()

In [7]:
len(cityid),len(cityid[cityid == 1]),len(cityid[cityid < 3])

(12107, 3773, 5495)

In [ ]:
将出现只有1和2次的city，用-1代替

In [7]:
city = list(cityid[cityid < 3].index)
for item in city:
    train_x['city'].replace(item,-1,inplace = True)

In [9]:
cityid = train_x['city'].value_counts()
len(cityid)

6613

In [ ]:
对test的操作

In [ ]:
首先把noroom去掉

In [8]:
test_A.drop(['noroom'],axis = 1,inplace = True)
test_B.drop(['noroom'],axis = 1,inplace = True)

In [9]:
test_A['zone'].replace(np.nan,-1,inplace = True)
test_A['zone'].replace(0,-1,inplace = True)
test_B['zone'].replace(np.nan,-1,inplace = True)
test_B['zone'].replace(0,-1,inplace = True)

In [ ]:
# 后边要用上，先别drop
# test_A.drop(['orderdate'],axis = 1,inplace = True)
# test_B.drop(['orderdate'],axis = 1,inplace = True)

In [ ]:
数据集分裂
train
831-913 914
test
901-914 915
这里需要注意两个验证集914和915：
各自的时间跨度为：901-914,902-915

In [10]:
from datetime import datetime
v831 = datetime.strptime('2017-08-31', '%Y-%m-%d')
v901 = datetime.strptime('2017-09-01', '%Y-%m-%d')
v902 = datetime.strptime('2017-09-02', '%Y-%m-%d')
v913 = datetime.strptime('2017-09-13', '%Y-%m-%d')
v914 = datetime.strptime('2017-09-14', '%Y-%m-%d')
v915 = datetime.strptime('2017-09-15', '%Y-%m-%d')

In [ ]:
train

In [11]:
t831913 = train_x[train_x['orderdate'] < v914]
t831913 = t831913[t831913['orderdate'] >= v831]
t901914 = train_x[train_x['orderdate'] < v915]
t901914 = t901914[t901914['orderdate'] >= v901]

In [ ]:
对应的test
注意时间上，即便是小于等于0615，也不会包括615的信息，所以可以理解为这个时间是0615 00:00，
因此小于等于的，都还是615之前的，不包括615这一天。

In [ ]:
处理y——放到最后吧，进行完特征挖掘后再处理y

In [ ]:
'''
y831913 = t831913['noroom']
t831913.drop(['noroom'],axis = 1,inplace = True)
y901914 = t901914['noroom']
t901914.drop(['noroom'],axis = 1,inplace = True)
'''

In [ ]:
还需要将orderdate也都drop掉——这个先等等

In [ ]:
# t901914.drop(['orderdate'],axis = 1,inplace = True)
# t831913.drop(['orderdate'],axis = 1,inplace = True)
# test914.drop(['orderdate'],axis = 1,inplace = True)

In [ ]:
对test的处理有些不同，可以放到后边处理好train之后，再处理test，尤其是914，区间正好就是901-914这个set上
可以将处理好的结果移植过来。

In [ ]:
test需要加上train_x的902到914部分，合成一个新的test，然后最后只要915这部分的。

In [12]:
# 获得自己是第几个订这个room的值
def get_room_top_x(train_x,v1,v2):
    # 获得初始数据
    temp = train_x[['orderdate','room']]
    # 按照时间分割数据
    tempa = temp['orderdate'] < v2 
    tempb = temp['orderdate'] >= v1
    tempc = tempa & tempb
    temp['group']= tempc.values
    # 按照room + 时间进一步分割数据
    table = temp.groupby(['room','group'])
    table = table.count().reset_index()
    table.columns = ['room','group','count_max']
    # 合并到初始表格中去
    temp = pd.merge(temp,table,how = 'left')
    # 需要对temp重新进行排序，便于后边给出正确的位次
    temp.sort_values(['room','orderdate'],ascending = False,inplace = True)  
    # 按照顺序给出位次
    temp_list = []
    for item in temp['count_max']:
        temp_list.append(item)
    for i in range(1,len(temp_list)):
        before = temp_list[i-1]
        if before == 1:
            continue
        else:
            temp_list[i] = before - 1
    temp['count_max'] = temp_list
    # 需要将index恢复成正确的，原来的形式，之后才可以给最初的表格
    temp.reset_index(inplace = True)
    temp.sort_values(['index'],inplace = True)
    temp.index = temp['index']
    # 将最终结果扔回train
    train_x['room_order_number'] = temp['count_max']
    return train_x

In [13]:
# 831-913
v907 = datetime.strptime('2017-09-07', '%Y-%m-%d')
# 按照最大到最小去排列
t831913.sort_values(['orderdate'],ascending = False,inplace = True)
# 重整index
t831913.reset_index(inplace = True)
t831913.drop(['index'],axis = 1,inplace = True)
t831913 = get_room_top_x(t831913,v907,v914)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
t831913[['orderdate','room','room_order_number']].head()

,orderdate,room,room_order_number
0,2017-09-13 23:59:57,413697,1
1,2017-09-13 23:59:56,29009193,5
2,2017-09-13 23:59:54,94768472,1
3,2017-09-13 23:59:50,20899290,1
4,2017-09-13 23:59:44,70523754,2


In [17]:
t831913.columns

Index(['city', 'countryid', 'glat', 'glon', 'hotel', 'hotelbelongto',
       'hotelstar', 'isebookinghtl', 'isholdroom', 'isvendor',
       'masterbasicroomid', 'masterhotelid', 'noroom', 'ordadvanceday',
       'orderdate', 'orderid', 'room', 'supplierchannel', 'supplierid',
       'wuli_totalrooms', 'zifangxing_totalrooms', 'zone', 'order_hour',
       'order_minute', 'day_stay', 'room_order_number'],
      dtype='object')

In [14]:
# 901-914
v908 = datetime.strptime('2017-09-08', '%Y-%m-%d')
# 按照最大到最小去排列
t901914.sort_values(['orderdate'],ascending = False,inplace = True)
# 重整index
t901914.reset_index(inplace = True)
t901914.drop(['index'],axis = 1,inplace = True)
t901914 = get_room_top_x(t901914,v908,v915)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
验证集914

In [15]:
test914 = t901914[t901914['orderdate'] >= v914]
y914 = test914['noroom']

In [ ]:
对于test则有些麻烦，不能直接取，
需要用902-915的来重构。

In [16]:
t902914 = train_x[train_x['orderdate'] < v915]
t902914 = t902914[t902914['orderdate'] >= v902]

In [17]:
# testA 902-915
test = pd.concat([test_A,t902914]) # 这里会导致test有noroom列，不过无所谓的后边删了就行
v909 = datetime.strptime('2017-09-09', '%Y-%m-%d')
v916 = datetime.strptime('2017-09-16', '%Y-%m-%d')
# 按照最大到最小去排列
test.sort_values(['orderdate'],ascending = False,inplace = True)
# 重整index
test.reset_index(inplace = True)
test.drop(['index'],axis = 1,inplace = True)
test = get_room_top_x(test,v909,v916)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
test_A = test[test['orderdate'] >= v915]

In [19]:
test_A.shape

(11035, 23)

In [ ]:
# testB

In [20]:
test = pd.concat([test_B,t902914])
v909 = datetime.strptime('2017-09-09', '%Y-%m-%d')
v916 = datetime.strptime('2017-09-16', '%Y-%m-%d')
# 按照最大到最小去排列
test.sort_values(['orderdate'],ascending = False,inplace = True)
# 重整index
test.reset_index(inplace = True)
test.drop(['index'],axis = 1,inplace = True)
test = get_room_top_x(test,v909,v916)
test_B = test[test['orderdate'] >= v915]

E:\anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
test_B.shape

(11036, 23)

In [ ]:
最后将没必要留下的列删除

In [ ]:
首先是各个x的noroom

In [22]:
y831913 = t831913['noroom']
t831913.drop(['noroom'],axis = 1,inplace = True)
y901914 = t901914['noroom']
t901914.drop(['noroom'],axis = 1,inplace = True)

test914.drop(['noroom'],axis = 1,inplace = True)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
t831913.shape,t901914.shape,test_A.shape,test_B.shape,test914.shape

((307564, 22), (309802, 22), (11035, 23), (11036, 23), (23846, 22))

In [24]:
y831913.shape,y901914.shape,y914.shape

((307564,), (309802,), (23846,))

In [25]:
t831913.columns

Index(['city', 'countryid', 'glat', 'glon', 'hotelbelongto', 'hotelstar',
       'isebookinghtl', 'isholdroom', 'isvendor', 'ordadvanceday', 'orderdate',
       'orderid', 'room', 'supplierchannel', 'supplierid', 'wuli_totalrooms',
       'zifangxing_totalrooms', 'zone', 'order_hour', 'order_minute',
       'day_stay', 'room_order_number'],
      dtype='object')

In [26]:
t901914.columns

Index(['city', 'countryid', 'glat', 'glon', 'hotelbelongto', 'hotelstar',
       'isebookinghtl', 'isholdroom', 'isvendor', 'ordadvanceday', 'orderdate',
       'orderid', 'room', 'supplierchannel', 'supplierid', 'wuli_totalrooms',
       'zifangxing_totalrooms', 'zone', 'order_hour', 'order_minute',
       'day_stay', 'room_order_number'],
      dtype='object')

In [27]:
test_A.columns

Index(['city', 'countryid', 'day_stay', 'glat', 'glon', 'hotelbelongto',
       'hotelstar', 'isebookinghtl', 'isholdroom', 'isvendor', 'noroom',
       'ordadvanceday', 'order_hour', 'order_minute', 'orderdate', 'orderid',
       'room', 'supplierchannel', 'supplierid', 'wuli_totalrooms',
       'zifangxing_totalrooms', 'zone', 'room_order_number'],
      dtype='object')

In [28]:
test_A.drop(['noroom'],axis = 1,inplace = True)
test_B.drop(['noroom'],axis = 1,inplace = True)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
test_A.drop(['orderdate'],axis = 1,inplace = True)
test_B.drop(['orderdate'],axis = 1,inplace = True)
test914.drop(['orderdate'],axis = 1,inplace = True)
t831913.drop(['orderdate'],axis = 1,inplace = True)
t901914.drop(['orderdate'],axis = 1,inplace = True)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
t831913.shape,t901914.shape,test_A.shape,test_B.shape,test914.shape

((307564, 21), (309802, 21), (11035, 21), (11036, 21), (23846, 21))

In [ ]:
存档

In [31]:
t831913.to_csv('dataset/final/t831913.csv',index = False)
test914.to_csv('dataset/final/test914.csv',index = False)
t901914.to_csv('dataset/final/t901914.csv',index = False)
test_A.to_csv('dataset/final/test_A.csv',index = False)
test_B.to_csv('dataset/final/test_B.csv',index = False)

In [32]:
y831913.to_csv('dataset/final/y831913.csv',index = False,)
y901914.to_csv('dataset/final/y901914.csv',index = False)
y914.to_csv('dataset/final/y914.csv',index = False)